# [196. Delete Duplicate Emails](https://leetcode.com/problems/delete-duplicate-emails/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Person

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| email       | varchar |
+-------------+---------+</pre>
id is the primary key (column with unique values) for this table.
Each row of this table contains an email. The emails will not contain uppercase letters.


Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.

For SQL users, please note that you are supposed to write a DELETE statement and not a SELECT one.

For Pandas users, please note that you are supposed to modify Person in place.

After running your script, the answer shown is the Person table. The driver will first compile and run your piece of code and then show the Person table. The final order of the Person table does not matter.

The result format is in the following example.



Example 1:

Input:
Person table:
<pre>+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+</pre>
Output:
<pre>+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+</pre>
Explanation: john@example.com is repeated two times. We keep the row with the smallest Id = 1.

In [4]:
# pandas schema

import pandas as pd

data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id': 'int64', 'email': 'object'})

# to spark dataframe

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

person_df = spark.createDataFrame(person)
person_df.show(truncate=False)

+---+----------------+
|id |email           |
+---+----------------+
|1  |john@example.com|
|2  |bob@example.com |
|3  |john@example.com|
+---+----------------+



In [27]:
from pyspark.sql import Window

# solving in spark dataframe

# spark does not support direct delete being OLAP system.
# One solution is to re-write the data after applying filter. (not done in this example)


person_df.alias('p1')\
          .join(person_df.alias('p2'),
                (col('p1.email')!=col('p2.email')) & (col('p1.id') < col('p2.id')),
                'inner')\
          .select('p1.*')\
          .show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+



In [32]:
# solving in spark SQL
#deleting not supported

person_df.createOrReplaceTempView('person')

spark.sql('''
delete from person
where id in(
select p1.id
from person p1 join person p2
on p1.email = p2.email and p1.id > p2.id)
''').show()


# mysql solution for OLTP:
'''
delete
from person
where id in (
    select
        id
    from
        (select id,row_number() over (partition by email order by id) as rn from person) p
    where p.rn>1
 );
'''

Py4JJavaError: An error occurred while calling o23.sql.
: org.apache.spark.SparkException: [INTERNAL_ERROR] Unexpected table relation: LogicalRDD [id#0L, email#1], false

	at org.apache.spark.SparkException$.internalError(SparkException.scala:92)
	at org.apache.spark.SparkException$.internalError(SparkException.scala:96)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Strategy.apply(DataSourceV2Strategy.scala:326)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:70)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.c

In [ ]:
spark.stop()